In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


In [2]:
add="ready_to_be_trained.csv"

df=pd.read_csv(add, header=0, sep=',')


In [3]:
df.drop(labels='Unnamed: 0', axis=1, inplace=True)


In [4]:
train=df[df.SeriousDlqin2yrs.notnull()]
test=df[df.SeriousDlqin2yrs.isnull()]

In [5]:
train.loc[:,'SeriousDlqin2yrs']=train['SeriousDlqin2yrs'].astype(int)

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,1:].values, train.SeriousDlqin2yrs.values, stratify=train.SeriousDlqin2yrs.values, test_size=0.33, random_state=42)

In [8]:
best_iterations=[]
best_scores=[]
for lr in [1,0.1,0.01,0.001]:
    model=XGBClassifier(n_estimators=20000, learning_rate=lr)
    eval_set=[(X_test,  y_test )]
    model.fit(X_train,y_train, early_stopping_rounds=30, eval_metric="logloss", eval_set=eval_set, verbose=False)
    best_iterations.append(model.best_iteration)
    best_scores.append(model.best_score)
    

In [9]:
best_scores

[0.180311, 0.177472, 0.177431, 0.177544]

In [10]:
best_iterations

[11, 236, 1764, 13892]

In [11]:
best_iterations=[]
best_scores=[]
for mf in [0.3,0.5,0.7,1]:
    model=XGBClassifier(n_estimators=3000,colsample_bytree=mf, learning_rate=0.01)
    eval_set=[(X_test,  y_test )]
    model.fit(X_train,y_train, early_stopping_rounds=30, eval_metric="logloss", eval_set=eval_set, verbose=False)
    best_iterations.append(model.best_iteration)
    best_scores.append(model.best_score)
    

In [12]:
best_iterations

[2219, 1862, 1759, 1764]

In [13]:
best_scores

[0.17744, 0.177374, 0.177398, 0.177431]

In [14]:
best_iterations=[]
best_scores=[]
for ss in [0.5,0.8,1]:
    model=XGBClassifier(n_estimators=3000,colsample_bytree=0.5, subsample=ss, learning_rate=0.01)
    eval_set=[(X_test,  y_test )]
    model.fit(X_train,y_train, early_stopping_rounds=30, eval_metric="logloss", eval_set=eval_set, verbose=False)
    best_iterations.append(model.best_iteration)
    best_scores.append(model.best_score)

In [15]:
best_iterations

[1525, 1905, 1862]

In [16]:
best_scores

[0.177174, 0.177096, 0.177374]

In [17]:
best_iterations=[]
best_scores=[]
for md in [1,3,6,9,12]:
    model=XGBClassifier(n_estimators=4000,colsample_bytree=0.5, subsample=0.8, max_depth=md, learning_rate=0.01)
    eval_set=[(X_test,  y_test )]
    model.fit(X_train,y_train, early_stopping_rounds=30, eval_metric="logloss", eval_set=eval_set, verbose=False)
    best_iterations.append(model.best_iteration)
    best_scores.append(model.best_score)

In [18]:
best_iterations

[3999, 1905, 930, 660, 546]

In [19]:
best_scores

[0.180535, 0.177096, 0.177015, 0.17761, 0.178727]

In [20]:
best_iterations=[]
best_scores=[]
for ni in [1,3,5,7,9]:
    model=XGBClassifier(n_estimators=3000,colsample_bytree=0.5, subsample=0.8, max_depth=6, min_child_weight=ni, learning_rate=0.01)
    eval_set=[(X_test,  y_test )]
    model.fit(X_train,y_train, early_stopping_rounds=30, eval_metric="logloss", eval_set=eval_set, verbose=False)
    best_iterations.append(model.best_iteration)
    best_scores.append(model.best_score)

In [21]:
best_iterations

[930, 930, 1018, 1000, 931]

In [22]:
best_scores

[0.177015, 0.17698, 0.176988, 0.176944, 0.177002]

In [28]:
model=XGBClassifier(n_estimators=1018,colsample_bytree=0.5, subsample=0.8, max_depth=6, min_child_weight=3, learning_rate=0.01)
model.fit(train.iloc[:,1:].values, train.SeriousDlqin2yrs.values)

preds=model.predict_proba(test.iloc[:,1:].values)
preds_df=pd.DataFrame({'Probability':preds[:,1]},index=range(1,101504))
preds_df.index.name='Id'
preds_df.to_csv('predictions3.csv')